<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#import-libraries" data-toc-modified-id="import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>import libraries</a></span></li><li><span><a href="#define-paths" data-toc-modified-id="define-paths-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>define paths</a></span></li><li><span><a href="#create-folders-in-test-and-train-directories-for-image-output" data-toc-modified-id="create-folders-in-test-and-train-directories-for-image-output-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>create folders in test and train directories for image output</a></span></li><li><span><a href="#gather-audio-files-for-processing" data-toc-modified-id="gather-audio-files-for-processing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>gather audio files for processing</a></span></li><li><span><a href="#wav-file-visualizations" data-toc-modified-id="wav-file-visualizations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>wav file visualizations</a></span></li><li><span><a href="#mel-spectograms" data-toc-modified-id="mel-spectograms-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>mel spectograms</a></span></li><li><span><a href="#onset-and-onset-strength-of-clip" data-toc-modified-id="onset-and-onset-strength-of-clip-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>onset and onset strength of clip</a></span></li><li><span><a href="#chromagrams" data-toc-modified-id="chromagrams-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>chromagrams</a></span></li><li><span><a href="#chroma-cqt" data-toc-modified-id="chroma-cqt-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>chroma cqt</a></span></li><li><span><a href="#mfccs" data-toc-modified-id="mfccs-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>mfccs</a></span></li></ul></div>

# import libraries

In [34]:
# directory modules
import os
from pathlib import Path
# audio modules
import audioread
import librosa
from librosa import feature
import librosa.display 
import soundfile 
# data wrangling modules
import pandas as pd 
import scipy
from scipy import stats
import geopandas as gpd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# general modules
import numpy as np
from tqdm import tqdm

In [4]:
wav_df = pd.read_csv('/Users/badmin/DSC/birdcall_project/data_100.csv')

In [5]:
filenames = list(wav_df.filename)
codes = list(wav_df.ebird_code)
file_code = dict(zip(filenames, codes))

# define paths

In [8]:
project = '/Users/badmin/DSC/birdcall_project/'
#image_path = '/Users/badmin/DSC/birdcall_project/wavdisplay100/'
audio_path = '/Users/badmin/DSC/birdcall_project/wav_audio_100/'

In [84]:
wav_df.head()

,Unnamed: 0,filename,season,tod,ebird_code,country,counts
0,24,XC102968.wav,summer,morning,warvir,United States,100
1,25,XC102969.wav,summer,morning,warvir,United States,100
2,27,XC102972.wav,summer,day,bnhcow,United States,100
3,29,XC104511.wav,summer,morning,daejun,United States,100
4,30,XC104512.wav,summer,morning,daejun,United States,100


In [38]:
#update classes variable and create files variable for north america bird df
classes = list(np.unique(wav_df.ebird_code))
files = list(np.unique(wav_df.filename))
print(len(files))
print(len(classes))

3200
32


# create folders in test and train directories for image output

In [39]:
image_path = '/Users/badmin/DSC/birdcall_project/test_images/'
for c in classes:
    os.mkdir(f'{image_path}{c}')

# gather audio files for processing

In [9]:
# gather all wav audio files
files = librosa.util.find_files(audio_path, ext=['wav']) 
filesArray = np.asarray(files)
filesArray

array(['/Users/badmin/DSC/birdcall_project/wav_audio_100/XC102968.wav',
       '/Users/badmin/DSC/birdcall_project/wav_audio_100/XC102969.wav',
       '/Users/badmin/DSC/birdcall_project/wav_audio_100/XC102972.wav',
       ...,
       '/Users/badmin/DSC/birdcall_project/wav_audio_100/XC93427.wav',
       '/Users/badmin/DSC/birdcall_project/wav_audio_100/XC99545.wav',
       '/Users/badmin/DSC/birdcall_project/wav_audio_100/XC99572.wav'],
      dtype='<U61')

In [50]:
files2df = list(filesArray)
files_df = pd.DataFrame(files2df)
files_df['filename'] = files2df
files_df['filename'].replace(regex=True, inplace=True, to_replace=r'/Users/badmin/DSC/birdcall_project/wav_audio_100/', value=r'')

In [51]:
files_df = files_df['filename']

In [10]:
data = pd.read_csv("data_100.csv")

In [11]:
bird_file = data.copy()
col = ['filename', 'ebird_code']
bird_file = bird_file[col]

In [12]:
bird_file.set_index('filename')
classes = list(bird_file.ebird_code)

In [13]:
for i in bird_file.index:
    bird_file['image_path'] = '/Users/badmin/DSC/birdcall_project/wavdisplay100/'+bird_file['ebird_code']+'/'

In [15]:
bird_file.head()

,filename,ebird_code,image_path
0,XC102968.wav,warvir,/Users/badmin/DSC/birdcall_project/wavdisplay1...
1,XC102969.wav,warvir,/Users/badmin/DSC/birdcall_project/wavdisplay1...
2,XC102972.wav,bnhcow,/Users/badmin/DSC/birdcall_project/wavdisplay1...
3,XC104511.wav,daejun,/Users/badmin/DSC/birdcall_project/wavdisplay1...
4,XC104512.wav,daejun,/Users/badmin/DSC/birdcall_project/wavdisplay1...


# wav file visualizations

In [40]:
image_paths = list(bird_file.image_path)
# create iterator
image_iter = iter(image_paths)

In [16]:
for path in tqdm(files):
   
        audio_file = str(path)
        path = f'{audio_path}/{audio_file}'
        img_path = next(image_iter)
        signal, sr = librosa.load(path, sr=32000)
        file_name = audio_file.rsplit( ".", 1 )[ 0 ]

        wavfig = plt.figure(figsize=(12,8))
       
        plt.plot(signal)
    
        plt.xlabel('Time')
        plt.ylabel('Frequency')
        wavfig.savefig(f'{img_path}{file_name}.png')


# mel spectograms

In [40]:
mels_have_df = pd.DataFrame()
mels_have_df['f'] = (os.listdir(mel_path))

mels_have_df['f'] = mels_have_df['f'].str.replace(r'png', "wav")
mels_have_df

,f
0,XC180252.wav
1,XC313482.wav
2,XC320048.wav
3,XC328414.wav
4,XC110127.wav
...,...
12345,XC147066.wav
12346,XC183742.wav
12347,XC29738.wav
12348,XC190058.wav


In [98]:
for i in bird_file.index:
    bird_file['image_path'] = '/Users/badmin/DSC/birdcall_project/chroma/'+bird_file['ebird_code']+'/'

In [99]:
bird_file

,filename,ebird_code,image_path
0,XC102968.wav,warvir,/Users/badmin/DSC/birdcall_project/mel100/warvir/
1,XC102969.wav,warvir,/Users/badmin/DSC/birdcall_project/mel100/warvir/
2,XC102972.wav,bnhcow,/Users/badmin/DSC/birdcall_project/mel100/bnhcow/
3,XC104511.wav,daejun,/Users/badmin/DSC/birdcall_project/mel100/daejun/
4,XC104512.wav,daejun,/Users/badmin/DSC/birdcall_project/mel100/daejun/
...,...,...,...
3195,XC81951.wav,swaspa,/Users/badmin/DSC/birdcall_project/mel100/swaspa/
3196,XC82756.wav,comgra,/Users/badmin/DSC/birdcall_project/mel100/comgra/
3197,XC93427.wav,gockin,/Users/badmin/DSC/birdcall_project/mel100/gockin/
3198,XC99545.wav,comgra,/Users/badmin/DSC/birdcall_project/mel100/comgra/


In [ ]:
image_paths = list(bird_file.image_path)
# create iterator
image_iter = iter(image_paths)

In [ ]:
mels_have_df = pd.DataFrame()
mels_have_df['f'] = (os.listdir(mel_path))

mels_have_df['f'] = mels_have_df['f'].str.replace(r'png', "wav")
mels_have_df

In [41]:
mels_have = list(mels_have_df['f'])

In [107]:
image_path = '/Users/badmin/DSC/birdcall_project/mels100/'
for c in classes:
    os.mkdir(f'{image_path}/{c}')

In [109]:
for f in tqdm(files):

        audio_file = f
        path =  f'{audio_path}/{audio_file}'
        img_path = next(image_iter)
      
        signal, sr = librosa.load(path, sr=32000, duration=30)
        file_name = audio_file.rsplit( ".", 1 )[ 0 ]
       
       

        S = librosa.feature.melspectrogram(y=signal,sr=32000,
                                                n_fft=2048,
                                                fmin = 100,
                                                fmax = 14000,
                                                n_mels=128, 
                                                window='hann', # hann window to minimize spectral leakage
                                                ) 

        melfig = plt.figure(figsize=(12, 8))
        librosa.display.specshow(librosa.power_to_db(S**2, ref=np.max))

      
        melfig.savefig(f'{img_path}{file_name}.png')
        plt.close(melfig);





  0%|          | 0/3200 [00:00<?, ?it/s]


  0%|          | 1/3200 [00:00<06:32,  8.16it/s]


  0%|          | 2/3200 [00:00<06:33,  8.12it/s]


  0%|          | 3/3200 [00:00<06:29,  8.20it/s]


  0%|          | 5/3200 [00:00<06:02,  8.80it/s]


  0%|          | 6/3200 [00:00<07:28,  7.12it/s]


  0%|          | 7/3200 [00:00<07:19,  7.26it/s]


  0%|          | 8/3200 [00:01<08:17,  6.41it/s]


  0%|          | 9/3200 [00:01<09:02,  5.88it/s]


  0%|          | 10/3200 [00:01<09:08,  5.82it/s]


  0%|          | 11/3200 [00:01<08:38,  6.14it/s]


  0%|          | 12/3200 [00:01<09:19,  5.70it/s]


  0%|          | 13/3200 [00:02<09:40,  5.49it/s]


  0%|          | 14/3200 [00:02<09:48,  5.42it/s]


  0%|          | 15/3200 [00:02<09:58,  5.32it/s]


  0%|          | 16/3200 [00:02<10:04,  5.27it/s]


  1%|          | 17/3200 [00:02<10:03,  5.28it/s]


  1%|          | 18/3200 [00:02<10:11,  5.20it/s]


  1%|          | 19/3200 [00:03<10:28,  5.06it/s]


  1%|          | 20/3200 [

 10%|▉         | 306/3200 [01:01<09:19,  5.17it/s]


 10%|▉         | 307/3200 [01:02<09:34,  5.04it/s]


 10%|▉         | 308/3200 [01:02<09:01,  5.34it/s]


 10%|▉         | 310/3200 [01:02<07:27,  6.45it/s]


 10%|▉         | 311/3200 [01:02<07:06,  6.78it/s]


 10%|▉         | 312/3200 [01:02<06:30,  7.40it/s]


 10%|▉         | 313/3200 [01:02<06:42,  7.18it/s]


 10%|▉         | 315/3200 [01:02<05:44,  8.38it/s]


 10%|▉         | 316/3200 [01:03<07:10,  6.70it/s]


 10%|▉         | 317/3200 [01:03<07:39,  6.28it/s]


 10%|▉         | 318/3200 [01:03<09:14,  5.20it/s]


 10%|▉         | 319/3200 [01:03<09:29,  5.06it/s]


 10%|█         | 320/3200 [01:03<09:18,  5.15it/s]


 10%|█         | 321/3200 [01:04<09:00,  5.33it/s]


 10%|█         | 322/3200 [01:04<09:08,  5.25it/s]


 10%|█         | 323/3200 [01:04<08:09,  5.88it/s]


 10%|█         | 325/3200 [01:04<07:00,  6.84it/s]


 10%|█         | 326/3200 [01:04<07:54,  6.06it/s]


 10%|█         | 327/3200 [01:05<08:53,  5.39i

 19%|█▉        | 610/3200 [01:53<05:10,  8.35it/s]


 19%|█▉        | 612/3200 [01:54<06:01,  7.16it/s]


 19%|█▉        | 614/3200 [01:54<05:59,  7.19it/s]


 19%|█▉        | 615/3200 [01:54<06:30,  6.62it/s]


 19%|█▉        | 616/3200 [01:54<06:21,  6.77it/s]


 19%|█▉        | 618/3200 [01:54<05:38,  7.64it/s]


 19%|█▉        | 619/3200 [01:54<05:40,  7.59it/s]


 19%|█▉        | 620/3200 [01:55<06:30,  6.60it/s]


 19%|█▉        | 621/3200 [01:55<06:51,  6.27it/s]


 19%|█▉        | 622/3200 [01:55<07:07,  6.03it/s]


 19%|█▉        | 623/3200 [01:55<07:20,  5.85it/s]


 20%|█▉        | 625/3200 [01:55<06:19,  6.78it/s]


 20%|█▉        | 626/3200 [01:56<06:56,  6.19it/s]


 20%|█▉        | 627/3200 [01:56<06:14,  6.87it/s]


 20%|█▉        | 628/3200 [01:56<06:31,  6.56it/s]


 20%|█▉        | 629/3200 [01:56<06:58,  6.15it/s]


 20%|█▉        | 630/3200 [01:56<10:22,  4.13it/s]


 20%|█▉        | 631/3200 [01:57<09:33,  4.48it/s]


 20%|█▉        | 633/3200 [01:57<08:16,  5.17i

 29%|██▊       | 917/3200 [02:42<05:12,  7.31it/s]


 29%|██▊       | 918/3200 [02:42<07:14,  5.25it/s]


 29%|██▊       | 919/3200 [02:43<07:16,  5.22it/s]


 29%|██▉       | 920/3200 [02:43<07:06,  5.35it/s]


 29%|██▉       | 921/3200 [02:43<07:03,  5.38it/s]


 29%|██▉       | 922/3200 [02:43<06:56,  5.47it/s]


 29%|██▉       | 923/3200 [02:43<07:00,  5.41it/s]


 29%|██▉       | 924/3200 [02:44<07:00,  5.41it/s]


 29%|██▉       | 925/3200 [02:44<07:01,  5.39it/s]


 29%|██▉       | 926/3200 [02:44<07:01,  5.39it/s]


 29%|██▉       | 927/3200 [02:44<07:01,  5.39it/s]


 29%|██▉       | 928/3200 [02:44<07:05,  5.35it/s]


 29%|██▉       | 929/3200 [02:44<07:11,  5.26it/s]


 29%|██▉       | 930/3200 [02:45<06:40,  5.67it/s]


 29%|██▉       | 931/3200 [02:45<06:11,  6.11it/s]


 29%|██▉       | 933/3200 [02:45<05:43,  6.60it/s]


 29%|██▉       | 934/3200 [02:45<06:01,  6.26it/s]


 29%|██▉       | 935/3200 [02:45<06:22,  5.92it/s]


 29%|██▉       | 936/3200 [02:46<06:31,  5.78i

 38%|███▊      | 1209/3200 [03:31<04:58,  6.66it/s]


 38%|███▊      | 1210/3200 [03:31<04:56,  6.72it/s]


 38%|███▊      | 1211/3200 [03:31<05:10,  6.40it/s]


 38%|███▊      | 1212/3200 [03:31<05:39,  5.86it/s]


 38%|███▊      | 1213/3200 [03:31<06:08,  5.39it/s]


 38%|███▊      | 1214/3200 [03:32<06:14,  5.31it/s]


 38%|███▊      | 1216/3200 [03:32<05:49,  5.68it/s]


 38%|███▊      | 1217/3200 [03:32<06:03,  5.45it/s]


 38%|███▊      | 1218/3200 [03:32<05:37,  5.88it/s]


 38%|███▊      | 1219/3200 [03:32<05:49,  5.67it/s]


 38%|███▊      | 1220/3200 [03:33<08:49,  3.74it/s]


 38%|███▊      | 1221/3200 [03:33<09:58,  3.31it/s]


 38%|███▊      | 1222/3200 [03:34<09:58,  3.30it/s]


 38%|███▊      | 1223/3200 [03:34<08:28,  3.89it/s]


 38%|███▊      | 1224/3200 [03:34<07:05,  4.64it/s]


 38%|███▊      | 1226/3200 [03:34<06:12,  5.29it/s]


 38%|███▊      | 1227/3200 [03:34<07:26,  4.42it/s]


 38%|███▊      | 1228/3200 [03:35<07:03,  4.66it/s]


 38%|███▊      | 1229/3200 [

...

 66%|██████▌   | 2114/3200 [06:02<03:14,  5.58it/s]


 66%|██████▌   | 2115/3200 [06:02<03:16,  5.51it/s]


 66%|██████▌   | 2116/3200 [06:02<02:59,  6.05it/s]


 66%|██████▌   | 2117/3200 [06:02<03:05,  5.82it/s]


 66%|██████▌   | 2118/3200 [06:03<03:08,  5.74it/s]


 66%|██████▌   | 2119/3200 [06:03<03:11,  5.64it/s]


 66%|██████▋   | 2120/3200 [06:03<03:13,  5.59it/s]


 66%|██████▋   | 2121/3200 [06:03<03:13,  5.57it/s]


 66%|██████▋   | 2122/3200 [06:03<03:16,  5.48it/s]


 66%|██████▋   | 2123/3200 [06:04<03:16,  5.49it/s]


 66%|██████▋   | 2124/3200 [06:04<03:18,  5.43it/s]


 66%|██████▋   | 2125/3200 [06:04<03:17,  5.43it/s]


 66%|██████▋   | 2126/3200 [06:04<03:19,  5.39it/s]


 66%|██████▋   | 2127/3200 [06:04<03:23,  5.27it/s]


 66%|██████▋   | 2128/3200 [06:04<03:23,  5.26it/s]


 67%|██████▋   | 2129/3200 [06:05<03:30,  5.09it/s]


 67%|██████▋   | 2130/3200 [06:05<03:13,  5.54it/s]


 67%|██████▋   | 2132/3200 [06:05<02:46,  6.43it/s]


 67%|██████▋   | 2133/3200 [

 76%|███████▌  | 2424/3200 [06:52<02:08,  6.02it/s]


 76%|███████▌  | 2425/3200 [06:52<01:59,  6.47it/s]


 76%|███████▌  | 2427/3200 [06:53<01:54,  6.73it/s]


 76%|███████▌  | 2428/3200 [06:53<01:49,  7.02it/s]


 76%|███████▌  | 2429/3200 [06:53<01:55,  6.69it/s]


 76%|███████▌  | 2430/3200 [06:53<01:46,  7.23it/s]


 76%|███████▌  | 2431/3200 [06:53<01:41,  7.61it/s]


 76%|███████▌  | 2432/3200 [06:53<02:01,  6.32it/s]


 76%|███████▌  | 2433/3200 [06:54<02:08,  5.96it/s]


 76%|███████▌  | 2434/3200 [06:54<02:13,  5.74it/s]


 76%|███████▌  | 2435/3200 [06:54<02:01,  6.32it/s]


 76%|███████▌  | 2437/3200 [06:54<01:40,  7.61it/s]


 76%|███████▌  | 2439/3200 [06:54<01:32,  8.25it/s]


 76%|███████▋  | 2440/3200 [06:54<01:34,  8.04it/s]


 76%|███████▋  | 2441/3200 [06:55<01:55,  6.56it/s]


 76%|███████▋  | 2442/3200 [06:55<02:07,  5.95it/s]


 76%|███████▋  | 2443/3200 [06:55<02:14,  5.63it/s]


 76%|███████▋  | 2445/3200 [06:55<01:48,  6.94it/s]


 76%|███████▋  | 2446/3200 [

 86%|████████▌ | 2737/3200 [07:40<00:56,  8.13it/s]


 86%|████████▌ | 2739/3200 [07:41<01:01,  7.53it/s]


 86%|████████▌ | 2741/3200 [07:41<01:01,  7.44it/s]


 86%|████████▌ | 2742/3200 [07:41<01:01,  7.40it/s]


 86%|████████▌ | 2743/3200 [07:41<01:07,  6.77it/s]


 86%|████████▌ | 2744/3200 [07:41<01:05,  7.01it/s]


 86%|████████▌ | 2746/3200 [07:42<01:04,  7.09it/s]


 86%|████████▌ | 2748/3200 [07:42<00:54,  8.32it/s]


 86%|████████▌ | 2749/3200 [07:42<00:59,  7.59it/s]


 86%|████████▌ | 2750/3200 [07:42<01:07,  6.71it/s]


 86%|████████▌ | 2751/3200 [07:45<06:47,  1.10it/s]


 86%|████████▌ | 2752/3200 [07:45<05:01,  1.48it/s]


 86%|████████▌ | 2754/3200 [07:45<03:50,  1.94it/s]


 86%|████████▌ | 2755/3200 [07:45<02:59,  2.48it/s]


 86%|████████▌ | 2756/3200 [07:45<02:33,  2.90it/s]


 86%|████████▌ | 2757/3200 [07:46<02:13,  3.32it/s]


 86%|████████▌ | 2758/3200 [07:46<01:56,  3.78it/s]


 86%|████████▋ | 2760/3200 [07:46<01:36,  4.57it/s]


 86%|████████▋ | 2761/3200 [

 95%|█████████▌| 3049/3200 [08:31<00:26,  5.77it/s]


 95%|█████████▌| 3050/3200 [08:31<00:23,  6.37it/s]


 95%|█████████▌| 3051/3200 [08:31<00:25,  5.89it/s]


 95%|█████████▌| 3052/3200 [08:31<00:22,  6.45it/s]


 95%|█████████▌| 3053/3200 [08:32<00:23,  6.30it/s]


 95%|█████████▌| 3054/3200 [08:32<00:24,  5.98it/s]


 95%|█████████▌| 3055/3200 [08:32<00:25,  5.76it/s]


 96%|█████████▌| 3056/3200 [08:32<00:25,  5.63it/s]


 96%|█████████▌| 3057/3200 [08:32<00:26,  5.44it/s]


 96%|█████████▌| 3058/3200 [08:33<00:26,  5.40it/s]


 96%|█████████▌| 3059/3200 [08:33<00:26,  5.39it/s]


 96%|█████████▌| 3060/3200 [08:33<00:25,  5.48it/s]


 96%|█████████▌| 3061/3200 [08:33<00:22,  6.20it/s]


 96%|█████████▌| 3063/3200 [08:33<00:20,  6.63it/s]


 96%|█████████▌| 3064/3200 [08:33<00:20,  6.54it/s]


 96%|█████████▌| 3065/3200 [08:34<00:22,  5.94it/s]


 96%|█████████▌| 3066/3200 [08:34<00:21,  6.11it/s]


 96%|█████████▌| 3067/3200 [08:34<00:20,  6.39it/s]


 96%|█████████▌| 3068/3200 [

StopIteration: 

# onset and onset strength of clip

In [ ]:
image_path = '/Users/badmin/DSC/birdcall_project/onset100/'
for c in classes:
    os.mkdir(f'{image_path}/{c}')

In [98]:
for i in bird_file.index:
    bird_file['image_path'] = '/Users/badmin/DSC/birdcall_project/onset100/'+bird_file['ebird_code']+'/'

In [99]:
bird_file

,filename,ebird_code,image_path
0,XC102968.wav,warvir,/Users/badmin/DSC/birdcall_project/mel100/warvir/
1,XC102969.wav,warvir,/Users/badmin/DSC/birdcall_project/mel100/warvir/
2,XC102972.wav,bnhcow,/Users/badmin/DSC/birdcall_project/mel100/bnhcow/
3,XC104511.wav,daejun,/Users/badmin/DSC/birdcall_project/mel100/daejun/
4,XC104512.wav,daejun,/Users/badmin/DSC/birdcall_project/mel100/daejun/
...,...,...,...
3195,XC81951.wav,swaspa,/Users/badmin/DSC/birdcall_project/mel100/swaspa/
3196,XC82756.wav,comgra,/Users/badmin/DSC/birdcall_project/mel100/comgra/
3197,XC93427.wav,gockin,/Users/badmin/DSC/birdcall_project/mel100/gockin/
3198,XC99545.wav,comgra,/Users/badmin/DSC/birdcall_project/mel100/comgra/


In [ ]:
image_paths = list(bird_file.image_path)
# create iterator
image_iter = iter(image_paths)

In [45]:
for f in tqdm(files):
    if f not in mels_have:
        audio_file = f
        path = f'{audio_path}/{audio_file}'
        signal, sr = librosa.load(path, sr=32000, duration=30)
        file_name = audio_file.rsplit( ".", 1 )[ 0 ]
        o_env = librosa.onset.onset_strength(y=signal, sr=sr)
        times = librosa.times_like(o_env, sr=sr)
        onset_frames = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr)
        onsetfig = plt.figure(figsize=(20, 10))
        plt.plot(times, o_env, label='Onset strength')
        plt.vlines(times[onset_frames], 0, o_env.max(), color='r', alpha=0.9,
           linestyle='--', label='Onsets')
        plt.legend()  
        onsetfig.savefig(f'onset/{file_name}.png')
        plt.close(onsetfig);

100%|██████████| 17692/17692 [18:53<00:00, 15.60it/s]   


# chromagrams

In [ ]:
image_path = '/Users/badmin/DSC/birdcall_project/chroma100/'
for c in classes:
    os.mkdir(f'{image_path}/{c}')

In [98]:
for i in bird_file.index:
    bird_file['image_path'] = '/Users/badmin/DSC/birdcall_project/chroma100/'+bird_file['ebird_code']+'/'

In [99]:
bird_file

,filename,ebird_code,image_path
0,XC102968.wav,warvir,/Users/badmin/DSC/birdcall_project/mel100/warvir/
1,XC102969.wav,warvir,/Users/badmin/DSC/birdcall_project/mel100/warvir/
2,XC102972.wav,bnhcow,/Users/badmin/DSC/birdcall_project/mel100/bnhcow/
3,XC104511.wav,daejun,/Users/badmin/DSC/birdcall_project/mel100/daejun/
4,XC104512.wav,daejun,/Users/badmin/DSC/birdcall_project/mel100/daejun/
...,...,...,...
3195,XC81951.wav,swaspa,/Users/badmin/DSC/birdcall_project/mel100/swaspa/
3196,XC82756.wav,comgra,/Users/badmin/DSC/birdcall_project/mel100/comgra/
3197,XC93427.wav,gockin,/Users/badmin/DSC/birdcall_project/mel100/gockin/
3198,XC99545.wav,comgra,/Users/badmin/DSC/birdcall_project/mel100/comgra/


In [ ]:
image_paths = list(bird_file.image_path)
# create iterator
image_iter = iter(image_paths)

In [ ]:
files1 = files[:10000]
files2 = files[10000:]

In [13]:
for f in tqdm(files):
        audio_file = f
        path = f'{audio_path}/{audio_file}'
        img_path = next(image_iter)
        signal, sr = librosa.load(path, sr=None, duration=30)
        
        file_name = audio_file.rsplit( ".", 1 )[ 0 ]

        S = np.abs(librosa.stft(signal))
        chroma = librosa.feature.chroma_stft(S=S, sr=sr)
        fig3 = plt.figure(figsize=(20,8))

        img = librosa.display.specshow(chroma, win_length=512, 
                            n_chroma=12,)

        fig3.savefig(f'{img_path}{file_name}.png')
        plt.close(fig3);



 73%|███████▎  | 12854/17692 [05:22<14:54,  5.41it/s]  /Users/badmin/opt/anaconda3/lib/python3.7/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
100%|██████████| 17692/17692 [17:55<00:00, 16.46it/s]  


# chroma cqt 

In [41]:
image_path = '/Users/badmin/DSC/birdcall_project/cqt100/'
for c in classes:
    os.mkdir(f'{image_path}/{c}')

In [42]:
for i in bird_file.index:
    bird_file['image_path'] = '/Users/badmin/DSC/birdcall_project/cqt100/'+bird_file['ebird_code']+'/'

In [43]:
bird_file

,filename,ebird_code,image_path
0,XC102968.wav,warvir,/Users/badmin/DSC/birdcall_project/cqt100/warvir/
1,XC102969.wav,warvir,/Users/badmin/DSC/birdcall_project/cqt100/warvir/
2,XC102972.wav,bnhcow,/Users/badmin/DSC/birdcall_project/cqt100/bnhcow/
3,XC104511.wav,daejun,/Users/badmin/DSC/birdcall_project/cqt100/daejun/
4,XC104512.wav,daejun,/Users/badmin/DSC/birdcall_project/cqt100/daejun/
...,...,...,...
3195,XC81951.wav,swaspa,/Users/badmin/DSC/birdcall_project/cqt100/swaspa/
3196,XC82756.wav,comgra,/Users/badmin/DSC/birdcall_project/cqt100/comgra/
3197,XC93427.wav,gockin,/Users/badmin/DSC/birdcall_project/cqt100/gockin/
3198,XC99545.wav,comgra,/Users/badmin/DSC/birdcall_project/cqt100/comgra/


In [44]:
image_paths = list(bird_file.image_path)
# create iterator
image_iter = iter(image_paths)

In [45]:
for f in tqdm(files):
        audio_file = f
        path = f'{audio_path}/{audio_file}'
        signal, sr = librosa.load(path, sr=32000, duration=30)
        img_path = next(image_iter)
        
        file_name = audio_file.rsplit( ".", 1 )[ 0 ]

        fig4 = plt.figure(figsize=(12,8))
        cq = librosa.feature.chroma_cqt(y=signal, sr=sr, n_chroma=12)
        img = librosa.display.specshow(cq, cmap='coolwarm')
        fig4.savefig(f'{img_path}{file_name}.png')
        plt.close(fig4);

            

  2%|▏         | 56/3200 [00:31<20:57,  2.50it/s]/Users/badmin/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=1008
  n_fft, y.shape[-1]
 55%|█████▍    | 1751/3200 [16:21<12:45,  1.89it/s]/Users/badmin/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
 99%|█████████▊| 3158/3200 [28:54<00:18,  2.27it/s]/Users/badmin/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=1019
  n_fft, y.shape[-1]
100%|██████████| 3200/3200 [29:14<00:00,  1.82it/s]


# mfccs

In [46]:
image_path = '/Users/badmin/DSC/birdcall_project/mfcc100/'
for c in classes:
    os.mkdir(f'{image_path}/{c}')

In [47]:
for i in bird_file.index:
    bird_file['image_path'] = '/Users/badmin/DSC/birdcall_project/mfcc100/'+bird_file['ebird_code']+'/'

In [48]:
bird_file

,filename,ebird_code,image_path
0,XC102968.wav,warvir,/Users/badmin/DSC/birdcall_project/mfcc100/war...
1,XC102969.wav,warvir,/Users/badmin/DSC/birdcall_project/mfcc100/war...
2,XC102972.wav,bnhcow,/Users/badmin/DSC/birdcall_project/mfcc100/bnh...
3,XC104511.wav,daejun,/Users/badmin/DSC/birdcall_project/mfcc100/dae...
4,XC104512.wav,daejun,/Users/badmin/DSC/birdcall_project/mfcc100/dae...
...,...,...,...
3195,XC81951.wav,swaspa,/Users/badmin/DSC/birdcall_project/mfcc100/swa...
3196,XC82756.wav,comgra,/Users/badmin/DSC/birdcall_project/mfcc100/com...
3197,XC93427.wav,gockin,/Users/badmin/DSC/birdcall_project/mfcc100/goc...
3198,XC99545.wav,comgra,/Users/badmin/DSC/birdcall_project/mfcc100/com...


In [49]:
image_paths = list(bird_file.image_path)
# create iterator
image_iter = iter(image_paths)

In [50]:
for f in tqdm(files):
        audio_file = f
        path = f'{audio_path}/{audio_file}'
        signal, sr = librosa.load(path, sr=32000, duration=30)
        img_path = next(image_iter)
        
        file_name = audio_file.rsplit( ".", 1 )[ 0 ]

        fig5 = plt.figure(figsize=(12,8))
        mfcc = librosa.feature.mfcc(y=signal, sr=sr, hop_length=2048, htk=True)
        img = librosa.display.specshow(mfcc)
        fig5.savefig(f'{img_path}{file_name}.png')
        plt.close(fig5);

            

100%|██████████| 3200/3200 [03:33<00:00, 14.99it/s]
